In [8]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext, filedialog
import serial
import serial.tools.list_ports
import threading
import time
import json
import pickle
from math import inf
import tkintermapview
from geographiclib.geodesic import Geodesic

# ==============================
# Configuración y constantes
# ==============================
SERIAL_PORT = None
SERIAL_BAUDRATE = 9600
ser = None
is_connected = False
last_telemetry_pos = None


# Protocolo de comunicación
HEADER = "GS#"
FOOTER = "#END"

# Modo de edición
EDIT_MODE = 'polygon'  # 'move', 'delete', 'home', 'polygon'

# ==============================
# Estructuras de datos
# ==============================
class MissionData:
    def __init__(self):
        self.polygon_points = []  # Puntos del polígono principal
        self.home_position = None  # Posición HOME
        self.altitude = 30         # Altitud de misión (m)
        self.spacing = 10          # Ancho entre pasadas (m)
     
    def to_dict(self):
        """Convierte a diccionario compacto para LoRa"""
        return {
            "p": [[round(lat, 6), round(lon, 6)] for lat, lon in self.polygon_points],
            "h": [round(self.home_position[0], 6), round(self.home_position[1], 6)] if self.home_position else None,
            "a": self.altitude,
            "s": self.spacing,
        }
    
    def from_dict(self, data):
        """Carga desde diccionario"""
        self.polygon_points = [tuple(point) for point in data.get("p", [])]
        self.home_position = tuple(data["h"]) if data.get("h") else None
        self.altitude = data.get("a", 30)
        self.spacing = data.get("s", 10)

# ==============================
# Funciones geodésicas con geographiclib
# ==============================
def calculate_distance(lat1, lon1, lat2, lon2):
    """Calcula la distancia entre dos puntos en metros"""
    return Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)['s12']

def calculate_bearing(lat1, lon1, lat2, lon2):
    """Calcula el rumbo entre dos puntos"""
    return Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)['azi1']

def get_point_at_distance(lat1, lon1, bearing, distance):
    """Obtiene un punto a una distancia y rumbo específicos"""
    result = Geodesic.WGS84.Direct(lat1, lon1, bearing, distance)
    return (result['lat2'], result['lon2'])

# ==============================
# Comunicación Serial
# ==============================
def connect_serial():
    global ser, is_connected, SERIAL_PORT
    SERIAL_PORT = port_var.get()
    
    if not SERIAL_PORT:
        messagebox.showerror("Error", "Selecciona un puerto serial primero")
        return False
    
    try:
        ser = serial.Serial(SERIAL_PORT, SERIAL_BAUDRATE, timeout=1)
        is_connected = True
        status_label.config(text=f"Conectado: {SERIAL_PORT}", fg="green")
        threading.Thread(target=receive_data, daemon=True).start()
        return True
    except Exception as e:
        messagebox.showerror("Error", f"No se pudo conectar: {str(e)}")
        return False

def disconnect_serial():
    global ser, is_connected
    if ser and ser.is_open:
        ser.close()
    is_connected = False
    status_label.config(text="Desconectado", fg="red")

def send_command(command_type, data=None):
    if not is_connected:
        messagebox.showwarning("Error", "No hay conexión serial")
        return
    
    message = {
        "t": command_type,
        "d": data,
        "ts": int(time.time() * 1000)
    }
    
    message_str = json.dumps(message)
    try:
        ser.write((message_str + "\n").encode())
        log_message(f"Enviado: {message_str}")
    except Exception as e:
        log_message(f"Error enviando: {str(e)}")

def receive_data():
    buffer = ""
    while is_connected and ser and ser.is_open:
        try:
            if ser.in_waiting > 0:
                data = ser.read(ser.in_waiting).decode(errors='ignore')
                buffer += data
                
                while HEADER in buffer and FOOTER in buffer:
                    start = buffer.find(HEADER) + len(HEADER)
                    end = buffer.find(FOOTER)
                    
                    if start < end:
                        message_str = buffer[start:end]
                        buffer = buffer[end + len(FOOTER):]
                        
                        try:
                            message = json.loads(message_str)
                            process_received_message(message)
                        except:
                            log_message(f"Mensaje inválido: {message_str}")
                    else:
                        break
            time.sleep(0.01)
        except Exception as e:
            log_message(f"Error recibiendo datos: {str(e)}")
            time.sleep(0.1)
    
    log_message("Hilo de recepción terminado")

def process_received_message(message):
    msg_type = message.get("t", "")
    data = message.get("d", {})
    
    if msg_type == "TELEMETRY":
        update_telemetry(data)
    elif msg_type == "STATUS":
        log_message(f"Estado: {data.get('message', '')}")
        status_message.set(f"Estado: {data.get('message', '')}")
    elif msg_type == "WP_REACHED":
        waypoint_reached(data.get("wp_number", 0))
    elif msg_type == "DEBUG":
        log_message(f"Debug: {data.get('message', '')}")
    else:
        log_message(f"Mensaje desconocido: {message}")

# ==============================
# Funciones de edición
# ==============================
def set_edit_mode(mode):
    global EDIT_MODE
    EDIT_MODE = mode
    
    modes = {
        'polygon': "Añadir Puntos de Polígono",
        'move': "Mover Marcadores",
        'delete': "Eliminar Marcadores"
    }
    
    status_message.set(f"Modo: {modes.get(mode, mode)}")
    log_message(f"Modo cambiado a: {modes.get(mode, mode)}")
    update_button_colors()

def update_button_colors():
    for btn in [add_btn, delete_btn]:
        btn.config(bg="SystemButtonFace")
    
    if EDIT_MODE == 'polygon':
        add_btn.config(bg="lightblue")
    elif EDIT_MODE == 'delete':
        delete_btn.config(bg="lightcoral")

def map_click(coords):
    lat, lon = coords
    
    if EDIT_MODE == 'polygon':
        current_mission.polygon_points.append((lat, lon))
        marker = mapa.set_marker(lat, lon, text=f"P{len(current_mission.polygon_points)}")
        polygon_markers.append(marker)
        update_polygon_drawing()
        log_message(f"Punto de polígono añadido: {lat:.6f}, {lon:.6f}")
        
            
    elif EDIT_MODE == 'delete':
        closest_marker = find_closest_marker(lat, lon)
        if closest_marker:
            delete_marker(closest_marker)
        else:
            log_message("No se encontró ningún marcador cercano")

def find_closest_marker(lat, lon, max_distance=1000):
    closest_marker = None
    min_distance = inf
    
    for i, marker in enumerate(polygon_markers):
        marker_pos = marker.position
        dist = calculate_distance(lat, lon, marker_pos[0], marker_pos[1])
        if dist < min_distance and dist < max_distance:
            min_distance = dist
            closest_marker = ('polygon', i, marker)
    
    if home_marker:
        marker_pos = home_marker.position
        dist = calculate_distance(lat, lon, marker_pos[0], marker_pos[1])
        if dist < min_distance and dist < max_distance:
            min_distance = dist
            closest_marker = ('home', 0, home_marker)
    
    return closest_marker

def delete_marker(marker_info):
    marker_type, index, marker = marker_info
    
    if marker_type == 'polygon':
       
            
        current_mission.polygon_points.pop(index)
        marker.delete()
        polygon_markers.pop(index)
        
        for i, marker in enumerate(polygon_markers):
            marker.set_text(f"P{i + 1}")
        
        update_polygon_drawing()
        log_message(f"Punto P{index + 1} eliminado")
        
    elif marker_type == 'home':
        home_marker.delete()
        home_marker = None
        current_mission.home_position = None
        home_position_var.set("")
        if home_line:
            home_line.delete()
            home_line = None
        log_message("Home position eliminada")

def set_home_position(lat, lon):
    global home_marker, home_line
    
    if home_marker:
        home_marker.delete()
    if home_line:
        home_line.delete()
    
    current_mission.home_position = (lat, lon)
    home_marker = mapa.set_marker(lat, lon, text="HOME", 
                                 marker_color_circle="green", marker_color_outside="green")
    home_position_var.set(f"{lat:.6f}, {lon:.6f}")
    log_message(f"Home position establecida: {lat:.6f}, {lon:.6f}")
    
    if current_mission.polygon_points:
        closest_point = find_closest_polygon_point(lat, lon)
        home_line = mapa.set_path([current_mission.home_position, closest_point], color="green", width=2)

def find_closest_polygon_point(lat, lon):
    if not current_mission.polygon_points:
        return None
    
    min_distance = inf
    closest_point = None
    
    for point in current_mission.polygon_points:
        dist = calculate_distance(lat, lon, point[0], point[1])
        if dist < min_distance:
            min_distance = dist
            closest_point = point
    
    return closest_point

# ==============================
# Funciones de dibujo
# ==============================
def update_polygon_drawing():
    global polygon_path
    if polygon_path:
        polygon_path.delete()
    if len(current_mission.polygon_points) >= 2:
        polygon_path = mapa.set_polygon(current_mission.polygon_points, 
                                      outline_color="blue", fill_color="lightblue", border_width=2)

# ==============================
# Gestión de misión
# ==============================
def update_mission_parameters():
    current_mission.altitude = altitude_var.get()
    current_mission.spacing = spacing_var.get()
    current_mission.takeoff_altitude = takeoff_altitude_var.get()
    log_message("Parámetros de misión actualizados")

def send_compact_mission():
    if len(current_mission.polygon_points) < 3:
        messagebox.showwarning("Error", "Definí al menos 3 puntos en el mapa")
        return
    
    if not current_mission.home_position:
        messagebox.showwarning("Error", "Establece una posición HOME primero")
        return
    
    mission_dict = current_mission.to_dict()
    send_command("MISSION_COMPACT", mission_dict)
    log_message("Misión enviada (formato compacto)")

def start_mission():
    send_compact_mission()

def return_home():
    send_command("RETURN")

def arm_drone():
    global last_telemetry_pos
    if not last_telemetry_pos:
        log_message("⚠️ No hay datos de telemetría para fijar HOME")
        return
    
    lat, lon = last_telemetry_pos
    set_home_position(lat, lon)
    log_message(f"🏠 HOME actualizado a telemetría: {lat:.6f}, {lon:.6f}")

    # Enviar comando de armado al dron
    send_command("ARM")
    log_message("🚁 Drone ARMADO")

    

def disarm_drone():
    global home_marker, home_line
    if home_marker:
        home_marker.delete()
        home_marker = None
    if home_line:
        home_line.delete()
        home_line = None
    current_mission.home_position = None
    home_position_var.set("")
    send_command("DISARM")
    log_message("🛑 Drone DESARMADO")


def reset_polygon():

    global current_mission, polygon_markers, polygon_path
    global home_marker, home_line, mission_path, waypoint_status

    # --- Borrar marcadores del polígono ---
    for m in polygon_markers:
        m.delete()
    polygon_markers.clear()

    # --- Borrar polígono azul (manual o cargado) ---
    if polygon_path is not None:
        try:
            polygon_path.delete()
        except Exception as e:
            log_message(f"Error borrando polígono: {e}")
        polygon_path = None

    # --- Borrar marcador HOME ---
    if home_marker:
        home_marker.delete()
        home_marker = None

    # --- Borrar línea hacia HOME ---
    if home_line:
        home_line.delete()
        home_line = None

    # --- Borrar trayectoria de misión ---
    if mission_path:
        mission_path.delete()
        mission_path = None

    # --- Resetear datos internos ---
    current_mission = MissionData()
    waypoint_status = []

    # --- Resetear GUI ---
    home_position_var.set("")
    progress_var.set("Waypoints: 0/0")
    progress_bar['value'] = 0
    altitude_var.set(30)
    spacing_var.set(10)

    log_message("🔄 Reset completo: polígono, HOME y misión eliminados")


def save_mission():
    if not current_mission.polygon_points:
        messagebox.showwarning("Error", "No hay misión para guardar")
        return
    
    filename = filedialog.asksaveasfilename(
        defaultextension=".mission",
        filetypes=[("Mission files", "*.mission"), ("All files", "*.*")]
    )
    
    if filename:
        try:
            with open(filename, 'wb') as f:
                pickle.dump(current_mission.to_dict(), f)
            log_message(f"Misión guardada: {filename}")
        except Exception as e:
            log_message(f"Error guardando misión: {str(e)}")

def load_mission():
    filename = filedialog.askopenfilename(
        filetypes=[("Mission files", "*.mission"), ("All files", "*.*")]
    )
    
    if filename:
        try:
            with open(filename, 'rb') as f:
                mission_data = pickle.load(f)
            
            reset_polygon()
            current_mission.from_dict(mission_data)
            update_interface_from_mission()
            log_message(f"Misión cargada: {filename}")
        except Exception as e:
            log_message(f"Error cargando misión: {str(e)}")

def update_interface_from_mission():
    # Restaurar polígono
    for i, (lat, lon) in enumerate(current_mission.polygon_points):
        marker = mapa.set_marker(lat, lon, text=f"P{i+1}")
        polygon_markers.append(marker)
    
    # Restaurar home
    if current_mission.home_position:
        set_home_position(current_mission.home_position[0], current_mission.home_position[1])
    
    # Restaurar parámetros
    altitude_var.set(current_mission.altitude)
    spacing_var.set(current_mission.spacing)
    takeoff_altitude_var.set(current_mission.takeoff_altitude)
    
    update_polygon_drawing()

# ==============================
# Funciones de UI
# ==============================
def log_message(message):
    log_text.config(state="normal")
    log_text.insert("end", f"{time.strftime('%H:%M:%S')} - {message}\n")
    log_text.see("end")
    log_text.config(state="disabled")

def update_telemetry(data):
    telemetry_text.config(state="normal")
    telemetry_text.delete(1.0, "end")
    
    telemetry_str = f"Latitud: {data.get('lat', 'N/A')}\n"
    telemetry_str += f"Longitud: {data.get('lon', 'N/A')}\n"
    telemetry_str += f"Altitud: {data.get('alt', 'N/A')} m\n"
    telemetry_str += f"Velocidad: {data.get('speed', 'N/A')} km/h\n"
    telemetry_str += f"Rumbo: {data.get('heading', 'N/A')}°\n"
    telemetry_str += f"Batería: {data.get('battery', 'N/A')}%\n"
    telemetry_str += f"Estado: {data.get('status', 'N/A')}\n"
    telemetry_str += f"Waypoint actual: {data.get('current_wp', 'N/A')}\n"
    telemetry_str += f"Waypoints totales: {data.get('total_wp', 'N/A')}\n"
    
    telemetry_text.insert("end", telemetry_str)
    telemetry_text.config(state="disabled")

    if 'lat' in data and 'lon' in data and data['lat'] != 'N/A' and data['lon'] != 'N/A':
        last_telemetry_pos = (data['lat'], data['lon'])
        update_drone_position(data['lat'], data['lon'])


def update_drone_position(lat, lon):
    if hasattr(update_drone_position, "marker") and update_drone_position.marker:
        update_drone_position.marker.delete()
    
    update_drone_position.marker = mapa.set_marker(lat, lon, text="DRON", 
                                                  marker_color_circle="red", marker_color_outside="red", 
                                                  text_color="red", font=("Helvetica", 12, "bold"))
    
    current_center = mapa.get_position()
    if calculate_distance(current_center[0], current_center[1], lat, lon) > 5000:
        mapa.set_position(lat, lon)
        mapa.set_zoom(15)

def initialize_waypoint_tracking(total_waypoints):
    global waypoint_status
    waypoint_status = ["pending"] * total_waypoints
    update_waypoint_display()

def waypoint_reached(wp_number):
    if 0 <= wp_number < len(waypoint_status):
        waypoint_status[wp_number] = "completed"
        update_waypoint_display()
        log_message(f"Waypoint {wp_number} completado")

def update_waypoint_display():
    if waypoint_status:
        completed = waypoint_status.count("completed")
        total = len(waypoint_status)
        progress_var.set(f"Waypoints: {completed}/{total}")
        progress_bar['value'] = (completed / total) * 100

# ==============================
# Interfaz gráfica
# ==============================
def create_gui():
    global root, mapa, status_label, telemetry_text, log_text
    global port_var, spacing_var, altitude_var, takeoff_altitude_var, status_message
    global home_position_var, progress_var, progress_bar
    global add_btn, home_btn, move_btn, delete_btn
    
    root = tk.Tk()
    root.title("Ground Station - Dron Buscador Térmico")
    root.geometry("1200x900")
    
    # Frame superior para conexión
    connection_frame = tk.Frame(root)
    connection_frame.pack(fill="x", padx=10, pady=5)
    
    ports = [port.device for port in serial.tools.list_ports.comports()]
    port_var = tk.StringVar(value=ports[0] if ports else "")
    
    tk.Label(connection_frame, text="Puerto:").pack(side="left")
    port_combo = ttk.Combobox(connection_frame, textvariable=port_var, values=ports, width=15)
    port_combo.pack(side="left", padx=5)
    
    def update_ports():
        new_ports = [port.device for port in serial.tools.list_ports.comports()]
        port_combo['values'] = new_ports
        if new_ports and not port_var.get():
            port_var.set(new_ports[0])
    
    tk.Button(connection_frame, text="Actualizar", command=update_ports).pack(side="left", padx=5)
    tk.Button(connection_frame, text="Conectar", command=connect_serial).pack(side="left", padx=5)
    tk.Button(connection_frame, text="Desconectar", command=disconnect_serial).pack(side="left", padx=5)
    
    status_label = tk.Label(connection_frame, text="Desconectado", fg="red")
    status_label.pack(side="right")
    
    # Frame de modos de edición
    edit_frame = tk.Frame(root)
    edit_frame.pack(fill="x", padx=10, pady=5)
    
    tk.Label(edit_frame, text="Modo Edición:", font=("Arial", 10, "bold")).pack(side="left")
    add_btn = tk.Button(edit_frame, text="Colocar marcadores", command=lambda: set_edit_mode('polygon'), width=20)
    add_btn.pack(side="left", padx=2)
    delete_btn = tk.Button(edit_frame, text="Eliminar marcadores", command=lambda: set_edit_mode('delete'), width=20)
    delete_btn.pack(side="left", padx=2)
    
    # Frame para parámetros
    params_frame = tk.Frame(root)
    params_frame.pack(fill="x", padx=10, pady=5)
    
    tk.Label(params_frame, text="Altitud (m):").pack(side="left")
    altitude_var = tk.IntVar(value=30)
    altitude_spin = tk.Spinbox(params_frame, from_=10, to=100, width=5, textvariable=altitude_var)
    altitude_spin.pack(side="left", padx=5)
    
    
    tk.Label(params_frame, text="Ancho pasadas (m):").pack(side="left", padx=(20, 5))
    spacing_var = tk.IntVar(value=10)
    spacing_spin = tk.Spinbox(params_frame, from_=5, to=50, width=5, textvariable=spacing_var)
    spacing_spin.pack(side="left", padx=5)
    
    home_position_var = tk.StringVar(value="")
    home_label = tk.Label(params_frame, textvariable=home_position_var, fg="green")
    home_label.pack(side="right")
    
    # Frame para estado y progreso
    status_frame = tk.Frame(root)
    status_frame.pack(fill="x", padx=10, pady=5)
    
    status_message = tk.StringVar(value="Modo: Polígono")
    status_display = tk.Label(status_frame, textvariable=status_message, fg="blue")
    status_display.pack(side="left")
    
    progress_var = tk.StringVar(value="Waypoints: 0/0")
    progress_label = tk.Label(status_frame, textvariable=progress_var)
    progress_label.pack(side="left", padx=20)
    
    progress_bar = ttk.Progressbar(status_frame, orient="horizontal", length=200, mode="determinate")
    progress_bar.pack(side="left", padx=5)
    
    # Frame para botones de misión
    mission_btn_frame = tk.Frame(root)
    mission_btn_frame.pack(fill="x", padx=10, pady=5)
    
    tk.Button(mission_btn_frame, text="COMENZAR MISIÓN", command=start_mission, bg="green", fg="white").pack(side="left", expand=True, padx=5)
    tk.Button(mission_btn_frame, text="REGRESAR", command=return_home, bg="orange", fg="white").pack(side="left", expand=True, padx=5)
    tk.Button(mission_btn_frame, text="RESET COMPLETO", command=reset_polygon, bg="red", fg="white").pack(side="left", expand=True, padx=5)
    tk.Button(mission_btn_frame, text="GUARDAR", command=save_mission, bg="blue", fg="white").pack(side="left", expand=True, padx=5)
    tk.Button(mission_btn_frame, text="CARGAR", command=load_mission, bg="purple", fg="white").pack(side="left", expand=True, padx=5)
    
    notebook = ttk.Notebook(root)
    notebook.pack(fill="both", expand=True, padx=10, pady=5)
    
    # Pestaña Misión
    frame_mission = ttk.Frame(notebook)
    notebook.add(frame_mission, text="Misión")
    
    global mapa
    mapa = tkintermapview.TkinterMapView(frame_mission, width=1000, height=500, corner_radius=0)
    mapa.pack(pady=10, fill="both", expand=True)
    mapa.set_position(-34.6037, -58.3816)
    mapa.set_zoom(12)
    mapa.add_left_click_map_command(map_click)
    
    # Pestaña Controles
    frame_controls = ttk.Frame(notebook)
    notebook.add(frame_controls, text="Controles")
    
    control_btn_frame = tk.Frame(frame_controls)
    control_btn_frame.pack(pady=10)
    
    tk.Button(control_btn_frame, text="ARMAR", command=arm_drone, bg="green", fg="white", width=15).pack(side="left", padx=10)
    tk.Button(control_btn_frame, text="DESARMAR", command=disarm_drone, bg="red", fg="white", width=15).pack(side="left", padx=10)
    
    # Pestaña Telemetría
    frame_telemetry = ttk.Frame(notebook)
    notebook.add(frame_telemetry, text="Telemetría")
    
    global telemetry_text
    telemetry_text = scrolledtext.ScrolledText(frame_telemetry, width=80, height=15)
    telemetry_text.pack(pady=10, fill="both", expand=True)
    telemetry_text.insert("end", "Esperando datos de telemetría...\n")
    telemetry_text.config(state="disabled")
    
    # Pestaña Log
    frame_log = ttk.Frame(notebook)
    notebook.add(frame_log, text="Log")
    
    global log_text
    log_text = scrolledtext.ScrolledText(frame_log, width=80, height=15)
    log_text.pack(pady=10, fill="both", expand=True)
    log_text.config(state="disabled")
    
    # Configurar modo inicial
    set_edit_mode('polygon')

# ==============================
# Variables globales
# ==============================
polygon_markers = []
polygon_path = None
home_marker = None
home_line = None
mission_path = None
waypoint_markers = []
waypoint_status = []
current_mission = MissionData()

# ==============================
# Inicialización
# ==============================
if __name__ == "__main__":
    create_gui()
    root.mainloop()